<a href="https://colab.research.google.com/github/GustavoHDLC1/Colab_Files_PADP/blob/main/Sem2_Practica6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [4]:
# lectura del archivo
df = pd.read_csv('/content/drive/MyDrive/MIAAD/PADP/titanic.csv')
df

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500
...,...,...,...,...,...,...,...,...
882,0,2,Rev. Juozas Montvila,male,27.0,0,0,13.0000
883,1,1,Miss. Margaret Edith Graham,female,19.0,0,0,30.0000
884,0,3,Miss. Catherine Helen Johnston,female,7.0,1,2,23.4500
885,1,1,Mr. Karl Howell Behr,male,26.0,0,0,30.0000


## Ejercicio 1: Análisis de la distribución de supervivencia por combinación de sexo y clase del pasajero.

In [43]:
aux = df.groupby(['Sex','Pclass'])['Survived'].mean().reset_index().sort_values(['Pclass', 'Sex'])
aux

,Sex,Pclass,Survived
0,female,1,0.968085
3,male,1,0.368852
1,female,2,0.921053
4,male,2,0.157407
2,female,3,0.500000
5,male,3,0.137026


In [45]:
# Identifica qué combinación tuvo la tasa de supervivencia más alta
aux[aux['Survived'] == aux['Survived'].max()]

,Sex,Pclass,Survived
0,female,1,0.968085


In [46]:
# Identifica qué combinación tuvo la tasa de supervivencia más baja.
aux[aux['Survived'] == aux['Survived'].min()]

,Sex,Pclass,Survived
5,male,3,0.137026


## Ejercicio 2: Identificación de familias grandes a bordo.

In [47]:
# Crea una nueva columna 'FamilySize' sumando las columnas 'SibSp' y 'Parch'.
df['FamilySize'] = df['Siblings/Spouses Aboard'] + df['Parents/Children Aboard']

In [51]:
# Considera como "familia grande" a aquellas donde 'FamilySize' es mayor a 3.
df['Familia_Grande'] = df['FamilySize'].apply(lambda x: 1 if x > 3 else 0)

In [53]:
# Calcula el número de pasajeros en familias grandes.
print(df[df['Familia_Grande']==1]['FamilySize'].sum())

354


In [55]:
# Calcula la proporción de supervivencia entre los pasajeros que pertenecen a familias grandes.
df.groupby('Familia_Grande')['Survived'].mean().reset_index()

,Familia_Grande,Survived
0,0,0.402424
1,1,0.161290


## Ejercicio 3: Segmentación por grupos de edad.
Clasifica a los pasajeros en las siguientes categorías de edad(tip puede resultar mas sencillo realizarlo con una función)

Menor de Edad

Mayor de Edad

In [56]:
df['Categoria_Edad'] = df['Age'].apply(lambda x: "Mayor de Edad" if x >= 18 else "Menor de Edad")

## Ejercicio 4: Comparación entre promedios calculados manualmente y con Pandas

In [61]:
import numpy as np

In [69]:
# Utiliza NumPy para calcular el promedio de las columnas 'Age' y 'Fare', ignorando valores nulos.
print('Edad promedio: ',np.nanmean(df['Age'].values))
print('Precio promedio: ',np.nanmean(df['Fare'].values))

Edad promedio:  29.471443066516347
Precio promedio:  32.30542018038331


In [70]:
# Compara estos valores con los promedios obtenidos utilizando los métodos nativos de Pandas.
print('Edad promedio',df['Age'].mean())
print('Precio promedio',df['Fare'].mean())

Edad promedio 29.471443066516347
Precio promedio 32.30542018038331


Verifica que los resultados sean consistentes. \
En este caso los valores son los mismos

## Ejercicio 5. Creación de intervalos de clase usando NumPy y análisis con Pandas

In [71]:
# Divide la columna 'Fare' en 5 intervalos equidistantes utilizando la función numpy.linspace
fare_min = df['Fare'].min()
fare_max = df['Fare'].max()
intervalos = np.linspace(fare_min, fare_max, 6)
print("Intervalos:", intervalos)

Intervalos: [  0.      102.46584 204.93168 307.39752 409.86336 512.3292 ]


In [72]:
# Crea una nueva columna en el DataFrame que asigne a cada pasajero el intervalo correspondiente de su tarifa
df['Fare_intervalo'] = pd.cut(df['Fare'], bins=intervalos, include_lowest=True)

In [80]:
# Calcula el número de pasajeros en cada intervalo utilizando Pandas y la proporción de supervivientes por intervalo.
df['Fare_intervalo'].value_counts().reset_index().sort_index()

,Fare_intervalo,count
0,"(-0.001, 102.466]",834
1,"(102.466, 204.932]",33
2,"(204.932, 307.398]",17
3,"(409.863, 512.329]",3
4,"(307.398, 409.863]",0


In [79]:
df.groupby('Fare_intervalo')['Survived'].mean().reset_index()

/tmp/ipython-input-3102306430.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('Fare_intervalo')['Survived'].mean().reset_index()


,Fare_intervalo,Survived
0,"(-0.001, 102.466]",0.363309
1,"(102.466, 204.932]",0.757576
2,"(204.932, 307.398]",0.647059
3,"(307.398, 409.863]",NaN
4,"(409.863, 512.329]",1.000000
